In [ ]:
# %pip install numpy
# %pip install matplotlib
# !docker run --name my-postgres -e POSTGRES_PASSWORD=mysecretpassword -p 5532:5432 -d postgres:latest
# %pip install psycopg2-binary

# docker run -it --rm postgres psql -h host.docker.internal -U <username> -d <database_name>
# %pip install sqlalchemy 
%pip install pony



In [ ]:
import psycopg2

# Define your database connection parameters
db_params = {
    "dbname": "postgres",
    "user": "postgres",        # Default superuser
    "password": "mysecretpassword",
    "host": "localhost",  # e.g., "localhost" or an IP address
    "port": "5532"   # e.g., "5432" (default port for PostgreSQL)
}

# Establish the connection
try:
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    print("Connection to PostgreSQL DB successful")

    # Optionally, execute a simple query to check the connection
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"PostgreSQL database version: {db_version}")

    # Close the cursor and connection
    cursor.close()
    connection.close()

except Exception as error:
    print(f"Error while connecting to PostgreSQL: {error}")


In [ ]:
from pony.orm import Database, Required, db_session, PrimaryKey
import psycopg2

# Define the database connection parameters
db = Database()

db.bind( provider='postgres', user='postgres', password='mysecretpassword', host='localhost', database='postgres', port='5532'  # default port is 5432
)

# Define the Product entity
class Product(db.Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str)
    price = Required(float)

# Generate the mapping
db.generate_mapping(create_tables=True)

# Insert data into the Product table
@db_session
def insert_product(name, price):
    Product(name=name, price=price)

# Example usage
def create_records():
    insert_product('p1', 10.99)
    insert_product('p2', 25.50)

    # Fetch and print all products
    with db_session:
        products = Product.select()
        for product in products:
            print(product.name, product.price)
@db_session
def delete_by_id(id):
  with db_session:
    Product.delete(p for p in Product if p.id == id)


In [ ]:
# create_records()

delete_by_id(2)

In [ ]:

recs = Product.select_by_sql("SELECT * FROM Product")
for r in recs:
  print(f"{r.id} {r.name}={r.price}")